# Первый скрипт. Бьет данные по контактам на хромосомы по РНК

Полное дерево директорий. Описано в доке.

In [1]:
subprocess.call('mkdir -p genomes/{Annotation/chr_features_hg_38, chr_sequence/chr_sequence, normalisation_data/{Fibro, K562}, reds_experiments}', shell = True)

NameError: name 'subprocess' is not defined

In [1]:
import pandas as pd
import requests
import io
from multiprocessing import Queue
from multiprocessing import Process
import subprocess 
from gtfparse import read_gtf
import time

Прописываются пути (через эксперимент и тип клеток)

In [18]:
experiment = 'GSM4041595_K562_rep2.hg38'
cell_type = 'K562'

In [19]:
table_name = '%s.tsv' % experiment
table_dir = '/Users/pitikov_egor/Documents/CourseWork/genomes/reds_experiments/%s/' % cell_type

Тут хранятся порезанные таблички по хромосомам

In [20]:
subprocess.call('mkdir %sparsed_contacts_%s' %(table_dir, table_name[:-4]), shell = True)

1

Данная функция открывает файл для записи и пишет туда старт, стоп и цепь всего контакта, не по 5' и 3' частям? а также в is_translate 1, если менее 50 нуклеотидов разниуа между контактом со стороны рнк и днк независимо от цепей (призвано отсеивать то, что почти наверняка только синтезировалост и поэтому свзалось - мб оасширить окно???).

Данные берет их очереди, куда будут докладываться контакты по мере чтения общей таблтцы основной программой.

Как старт берет наименьший из стартов на 5' и 3' концах, как стоп - нибольший стоп - это оказалось быстрее, чем проаисывать условтя для цери рнк. Как цепь колонку 5' у рнк - вроде должна с ней совпадать

Да, это можно загнать в awk, но оно и так не то, чтобы медленное и я боюсь лажи с awk.

In [21]:
def open_cut(outfile_name, contact_queue):
    output = open(outfile_name, 'w+')
    output.write('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n' % ('id', 'rna_chr', 'rna_bgn', 'rna_end', 'rna_strand', 'dna_chr', 'dna_bgn', 'dna_end', 'dna_strand', 'is_translate'))
    while True:
        contact_data = contact_queue.get()
        translate_status = 0
        if type(contact_data)==type(False):
            break
        if (contact_data[1]==contact_data[11]) and (abs(min(int(contact_data[2]), int(contact_data[7])) - int(contact_data[12]))<=500000) and (abs(max(int(contact_data[3]), int(contact_data[8])) - int(contact_data[13]))<=500000):
            translate_status = 1
        output.write('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n' % (contact_data[0], contact_data[1], min(int(contact_data[2]), int(contact_data[7])), max(int(contact_data[3]), int(contact_data[8])), contact_data[9], contact_data[11], contact_data[12], contact_data[13], contact_data[14], translate_status))
    output.close()
    

Делает массив с именами хромосом, очередь и процесс на каждую хромосому. Целевая для процесса open_cut, в качестве агументнов очередь на данную хромосому

In [22]:
chr_names=['chrMT', 'chrX', 'chrY']
for i in range(1, 23):
    chr_names.append('chr%d' % (i))

    

    
    
if __name__ == '__main__':
    for x in chr_names: 
        globals()['queue_%s' % x] = Queue()
        globals()['process_%s' % x] = Process(target=open_cut, args=('%sparsed_contacts_%s/%s_%s' % (table_dir, table_name[:-4], x, table_name), eval('queue_%s' % x)), name=x)
        eval('process_%s' % x).start()

In [23]:
redc_table = open(table_dir + table_name, 'r')

Построчно читает таблицу с контактами, в зависимости от хромосомы рнк  контакте пинает в нужную очередь, очрежи уже обрабатываются запущеннвыми выше функциями.

P. S. В табдицах разный разделителт в заголовек и в таблице - это норма?

In [24]:
line = redc_table.readline()


if __name__ == '__main__':
    while line:
        line = redc_table.readline()
        if line!='':
            splitsrtring = str(line).split(' ')
            eval('queue_%s' % splitsrtring[1]).put(splitsrtring)

    for x in chr_names:
        eval('queue_%s' % x).put(False)
    for x in chr_names:
        eval('process_%s' % x).join()



In [25]:
redc_table.close()